In [39]:
from test_data import gapped_sequences, sequences
from cogent.core.sequence import NucleicAcidSequence, DnaSequence
from cogent import DNA
from benchmark import Benchmark
cogent_benchmark = Benchmark("pycogent")

cogent_sequences = [DnaSequence(seq) for _, seq in sequences]
cogent_gapped_sequences = [DnaSequence(seq) for _, seq in gapped_sequences]

In [40]:
@cogent_benchmark
def object_creation():
    for id_, seq in sequences:
        DnaSequence(seq)

100 loops, best of 3: 3.17 ms per loop


In [41]:
@cogent_benchmark
def object_creation_validate():
    for id_, seq in sequences:
        DNA.makeSequence(seq)

100 loops, best of 3: 3.24 ms per loop


In [42]:
@cogent_benchmark
def reverse_complement():
    for e in cogent_sequences:
        DnaSequence(e).rc()

100 loops, best of 3: 13.7 ms per loop


In [43]:
@cogent_benchmark
def degap_all():
    for e in cogent_gapped_sequences:
        DnaSequence(e).degap()

100 loops, best of 3: 15.9 ms per loop


In [44]:
DnaSequence(gapped_sequences[0][1]).rc()

DnaSequence(-------... 7682)

In [45]:
cogent_benchmark.record("cogent.csv")